## Analyse de rentabilisation des livres audio

## Description du Projet

Les données d'une application Audiobook vous sont fournies. Logiquement, il se rapporte aux versions audio des livres SEULEMENT. Chaque client dans la base de données a effectué un achat au moins une fois, c'est pourquoi il est dans la base de données. Nous voulons créer un algorithme d'apprentissage machine basé sur nos données disponibles qui permet de prédire si un client achètera à nouveau un livre audio ou pas.
<br /> La tâche est simple : créer un algorithme d'apprentissage machine, qui est capable de prédire si un client va acheter à nouveau.
<br /> Il s'agit d'un problème de **classification** avec deux classes : ne veut pas acheter et achète, représenté par 0 et 1.



## L'utilité du projet du coté Marketing (Prenons "AudioBook" comme exemple d'une socièté)

L'idée principale est que si un client a une faible probabilité de revenir, il n'y a aucune raison de dépenser de l'argent en publicité auprès de lui. Si nous pouvons concentrer nos efforts UNIQUEMENT sur les clients qui sont susceptibles de se reconvertir, nous pouvons réaliser d'importantes économies. De plus, ce modèle permet d'identifier les paramètres les plus importants pour qu'un client revienne. L'identification de nouveaux clients crée de la valeur et des opportunités de croissance.




## Description des variables (features)

 Vous avez un fichier.csv qui résume les données. Il y a plusieurs variables : <br /> **Customer ID** : l'identifiant de chaque client <br /> **Longueur totale du livre** : somme des minutes de tous les achats <br /> **Longueur moyenne du livre** : longueur moyenne en minutes de tous les achats <br /> **Prix payé_global** : somme de tous les achats <br /> **Moyenne des prix payés** : moyenne de tous les achats<br /> **La revue** : (variable booléenne) si le client a quitté un examen <br /> **Evaluation du produit sur 10** : si le client a quitté un examen, examen sur 10 <br /> **Total de minutes écoutées** : nombre total des minutes écoutés par le client  <br /> **Complétion** = Total de minutes écoutées/Longueur totale du livre (variable numérique entre 0 et 1)<br /> **support technique** : nombre de demandes de support technique ,tout, du mot de passe oublié à l'assistance pour l'utilisation de l'application  <br />  **la date de la dernière visite moins la date d'achat (en jours)** : C'est une variable importante qui sert à préciser si un client prend le temps lors de son achat (si 0 alors le client n'a pas joué l'audio qu'il a acheté.<br /> Ce sont les entrées (à l'exclusion de l'ID client, car il est totalement arbitraire. C'est plus un nom qu'un numéro).<br /> **Targets(Les cibles)** : sont une variable booléenne (0 ou 1). Nous prenons une période de 2 ans dans nos entrées, et les 6 prochains mois comme objectifs. Ainsi, en fait, nous prédisons si : sur la base des 2 dernières années d'activité et d'engagement, un client va se convertir dans les 6 prochains mois. 6 mois, c'est un délai raisonnable. S'ils ne se convertissent pas après 6 mois, il y a de fortes chances qu'ils soient allés chez un concurrent ou qu'ils n'aient pas aimé la façon dont Audiobook digère l'information.
 

## Chargement des données 

In [1]:
import numpy as np
import pandas as pd
## Nous utiliserons la bibliothèque de prétraitement sklearn, car il sera plus facile de standardiser les données.
from sklearn import preprocessing

# Charger les données
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')


# Les entrées sont toutes les colonnes du csv, à l'exception de la première [ :,0].
# (qui n'est que l'identification arbitraire des clients qui ne portent aucune information utile),
# et le dernier [ :,-1] (qui est notre cible)

unscaled_inputs_all = raw_csv_data[:,1:-1]

# Les cibles sont dans la dernière colonne.
targets_all = raw_csv_data[:,-1]


## Blancer Les données

In [2]:
# Comptez combien de cibles sont 1 (ce qui signifie que le client a converti).
num_one_targets=int(np.sum(targets_all))

# Définir un compteur pour les cibles qui sont à 0 (ce qui signifie que le client n'a pas converti)
zero_targets_counter = 0

# Nous voulons créer un ensemble de données "équilibré", nous devrons donc supprimer certaines paires entrées/cibles.
# Déclarez une variable qui le fera :
indices_to_remove = []

# Comptez le nombre de cibles qui sont à 0. 
# Une fois qu'il y a autant de 0 que de 1, marquez les entrées où la cible est 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
# Créez deux nouvelles variables, une qui contiendra les entrées et une autre qui contiendra les cibles.
# Nous supprimons tous les index que nous avons marqués "supprimer" dans la boucle ci-dessus.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardiser les entrées

In [3]:
# C'est le seul endroit où nous utilisons la fonctionnalité sklearn. Nous tirerons parti de ses capacités de prétraitement
# C'est une simple ligne de code, qui standardise les entrées, comme nous l'avons expliqué dans l'une des conférences.
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Mélanger les données

In [5]:
# Quand les données ont été collectées, elles ont été classées par date
# Mélangez les index des données, de sorte que les données ne sont pas organisées de quelque façon que ce soit lorsque nous les alimentons.
# Puisque nous allons mettre en lots, nous voulons que les données soient réparties aussi aléatoirement que possible

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Utilisez les index mélangés pour mélanger les entrées et les cibles.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Diviser les données en Entrainement, Validation et Test

In [6]:
# Comptez le nombre total d'échantillons
samples_count = shuffled_inputs.shape[0]

# Comptez les échantillons dans chaque sous-ensemble, en supposant que nous voulons une distribution 80-10-10 de l'entrainement', de la validation et du test.
# Naturellement, les nombres sont entiers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# L'ensemble de données'test' contient toutes les données restantes.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Créer des variables qui enregistrent les entrées et les cibles pour l'entrainement
# Dans notre ensemble de données mélangées, ce sont les premières observations "train_samples_count".
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Créer des variables qui enregistrent les entrées et les cibles à valider.
# Ce sont les prochaines observations "validation_samples_count", suivant le "train_samples_count" que nous avons déjà assigné
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Créer des variables qui enregistrent les entrées et les cibles pour le test.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


#Retourner le nombre de cibles qui sont 1s, le nombre total d'échantillons et la proportion pour l'entraînement, la validation et le test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)




1789.0 3579 0.49986029617211514
211.0 447 0.4720357941834452
237.0 448 0.5290178571428571


### Sauvegardez les trois ensembles de données dans *.npz


In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

## Créez une classe qui regroupera les données



In [8]:
# Créez une classe qui fera le batching pour l'algorithme

class Audiobooks_Data_Reader():
    # L'ensemble de données est un arugment obligatoire, alors que la taille du lot est facultative.
    def __init__(self, dataset, batch_size = None):
    
        # L'ensemble de données qui charge en est un de "train", "validation", "test".
        # Par exemple, si j'appelle cette classe avec x('train',5), elle chargera 'Audiobooks_data_train.npz' avec une taille de lot de 5.

        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        # Deux variables qui prennent les valeurs des entrées et des cibles. Les entrées sont des flotteurs, les cibles sont des entiers
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Compte le numéro de lot, compte tenu de la taille que vous lui donnerez plus tard
        # Si la taille du lot est Aucune, nous validons ou testons, donc nous voulons prendre les données dans un seul lot.
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # la méthode pour charger le lot suivant
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        # Vous coupez l'ensemble de données par lots et la fonction "next" les charge l'un après l'autre.
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        # One-hot encoder les cibles.
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        # La fonction retournera le lot d'entrées et les cibles codées à un chaud.

        return inputs_batch, targets_one_hot
    
        
    # Une méthode nécessaire pour l'itération sur les lots, car nous allons les mettre en boucle
    # Cela indique à Python que la classe que nous définissons est itérable, c'est-à-dire que nous pouvons l'utiliser comme :
    # pour l'entrée, la sortie dans les données : 
        # do things
    # Un itérateur en Python est une classe avec une méthode __next__ qui définit exactement comment itérer à travers ses objets

    def __iter__(self):
        return self

## Créer l'algorithme d'apprentissage machine



## La Configuration des hyperparamètres

In [9]:
import tensorflow as tf
# La taille de l'entrée dépend du nombre de variables d'entrée. Nous en avons 10
input_size = 10
# Output size is 2, as we one-hot encoded the targets.
output_size = 2
# Choisissez une taille de couche cachée
hidden_layer_size = 100

# Réinitialisez le graphique par défaut, afin de pouvoir modifier les hyperparamètres, puis réexécuter le code.
tf.reset_default_graph()

C:\Users\Chhimy\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Créez les espaces réservés


In [10]:
inputs = tf.placeholder(tf.float32,[None, input_size])
targets = tf.placeholder(tf.int32,[None, output_size])

### description du modèle

Nous allons créer un réseau avec 2 calques cachés


In [11]:
# Poids et biais pour la première combinaison linéaire entre les entrées et la première couche cachée.
# la méthode get_variable pour utiliser l'initialisateur par défaut de TensorFlow qui est Xavier.
weights_1 = tf.get_variable("weights_1",[input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])
# pour la sortie nous utilisons la fonction d'activation ReLu 
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2",[hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])
# pour la sortie nous utilisons la fonction d'activation Sigmoid
outputs_2 = tf.nn.sigmoid(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3",[hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3",[output_size])
# Nous allons incorporer l'activation softmax dans la perte, 
outputs = tf.matmul(outputs_2, weights_3) + biases_3

### the loss function (la fonction de perte)

In [12]:
# Nous allons incorporer l'activation softmax dans la perte, 
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### La mésure de la prècision


la précision est le facteur qui détermine la puissance de notre modèle puisque nous allons la calculer sur le jeu de données d'entrainement (Train) et tester sa performance sur le jeu de données de test(Test).

In [13]:
# Obtenir un 0 ou un 1 pour chaque entrée indiquant si elle fournit la bonne réponse.
out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

### L'Etape d'Optimisation


In [14]:
# Optimize with Adam
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)



### lancement d'une session


pour démarrer le programme.

In [15]:
# Créez une session
sess = tf.InteractiveSession()

# Initialiser les variables
initializer = tf.global_variables_initializer()
sess.run(initializer)

# l'entrainement du modèle (Train)

In [16]:
# Choisir la taille du lot
batch_size = 100

# les points d'arret de l'algorithme
max_epochs = 100
prev_validation_loss = 999999999.

# Charger le premier lot de formation et de validation, en utilisant la classe que nous avons créée. 
# Les arguments se terminent par'Audiobooks_Data_<...>', où pour <...> on entre'train','validation' ou'test'.
train_data = Audiobooks_Data_Reader('train', batch_size)
validation_data = Audiobooks_Data_Reader('validation')

# créer une boucle pour les époques
for epoch_counter in range(max_epochs) :
    
    # la mise de l'époque loss en 0.
    curr_epoch_loss = 0.
    
    # Itérer sur les données d'entraînement 
    # Puisque train_data est une instance de la classe Audiobooks_Data_Reader,
    # nous pouvons itérer à travers elle en utilisant implicitement la méthode __next__ que nous avons définie ci-dessus.
    # Pour rappel, il regroupe les échantillons en lots, code les cibles en une seule fois et renvoie les résultats.
    # Entrées et cibles lot par lot Nombre d'entrées et de cibles lot par lot
    for input_batch, target_batch in train_data :
        _,batch_loss = sess.run([optimizer, mean_loss], 
                       feed_dict={inputs : input_batch, targets : target_batch})
        
        #Enregistrer la perte de lot dans la perte d'époque actuelle.
        curr_epoch_loss += batch_loss
    
    # trouver le moyen curr_epoch_loss.
    # batch_count est une variable, définie dans la classe Audiobooks_Data_Reader
    curr_epoch_loss /= train_data.batch_count
    
    # Définir la perte de validation et la précision de l'époque à zéro
    validation_loss = 0.
    validation_accuracy = 0.
    
    # Utiliser la même logique que le code pour propager l'ensemble de validation
    # Il n'y aura qu'un seul lot, car la classe a été créée de cette façon
    for input_batch, target_batch in validation_data :
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                               feed_dict={inputs : input_batch, targets : target_batch})
    
    # Imprimer les statistiques pour l'époque actuelle
    print('Epoque'+str(epoch_counter+1)+
          '. Perte entraînement: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Perte de validation: '+'{0:.3f}'.format(validation_loss)+
          '. Précision de validation: '+'{0:.2f}'.format(validation_accuracy  * 100.)+'%')
    
    
    
    # Déclenchement de l'arrêt anticipé si la perte de validation commence à augmenter.
    if validation_loss > prev_validation_loss :
        
          break
        
    # Mémoriser la perte de validation de cette époque pour l'utiliser comme précédemment lors de la prochaine itération.
    prev_validation_loss = validation_loss 
    
print("Fin de l'entrainement.")

Epoque1. Perte entraînement: 0.626. Perte de validation: 0.543. Précision de validation: 69.35%
Epoque2. Perte entraînement: 0.488. Perte de validation: 0.455. Précision de validation: 78.75%
Epoque3. Perte entraînement: 0.428. Perte de validation: 0.414. Précision de validation: 79.87%
Epoque4. Perte entraînement: 0.397. Perte de validation: 0.391. Précision de validation: 80.76%
Epoque5. Perte entraînement: 0.379. Perte de validation: 0.377. Précision de validation: 79.64%
Epoque6. Perte entraînement: 0.367. Perte de validation: 0.367. Précision de validation: 79.87%
Epoque7. Perte entraînement: 0.358. Perte de validation: 0.361. Précision de validation: 80.54%
Epoque8. Perte entraînement: 0.352. Perte de validation: 0.356. Précision de validation: 81.21%
Epoque9. Perte entraînement: 0.347. Perte de validation: 0.352. Précision de validation: 81.66%
Epoque10. Perte entraînement: 0.343. Perte de validation: 0.348. Précision de validation: 81.88%
Epoque11. Perte entraînement: 0.340. Pe

Epoque88. Perte entraînement: 0.306. Perte de validation: 0.312. Précision de validation: 82.55%
Epoque89. Perte entraînement: 0.305. Perte de validation: 0.312. Précision de validation: 82.77%
Epoque90. Perte entraînement: 0.305. Perte de validation: 0.312. Précision de validation: 82.77%
Epoque91. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 82.77%
Epoque92. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 83.00%
Epoque93. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 83.00%
Epoque94. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 83.00%
Epoque95. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 83.00%
Epoque96. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 83.00%
Epoque97. Perte entraînement: 0.305. Perte de validation: 0.311. Précision de validation: 83.00%
Fin de l'entrainement.


# Le Test du modèle

In [17]:
# Charger les données de test, en suivant la même logique que pour les données train_data et les données de validation.
test_data = Audiobooks_Data_Reader('test')

# Se propager vers l'avant à travers l'ensemble d'entraînement. Cette fois, nous n'avons besoin que de la précision

for inputs_batch, targets_batch in test_data:
    test_accuracy = sess.run([accuracy],
                     feed_dict={inputs: inputs_batch, targets: targets_batch})

#obtenir la précision du test en pourcentage
# Quand sess.run a une seule sortie, nous obtenons une liste (c'est ainsi qu'elle a été codée par Google), plutôt qu'un flotteur.
# Par conséquent, nous devons prendre la première valeur de la liste (la valeur à la position 0)
test_accuracy_percent = test_accuracy[0] * 100.

# afficher la précision du test
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 80.36%


La précision du modèle est de 80 % donc si on prend par exemple 100 clients qui ont achetés les livres audio le modèle peut prédire le comportement d'achat exactes des 80 clients(80 %)

**Décision Stratégique :** Comme une solution stratégique de la société AudioBook, elle doit se concentrer ces clients en leurs données des offres spéciales par exemple des réductions sur les achats(coupons), des cartes de fidélités car ils représentent le potentiel de d'achat pour AudioBook et c'est sûr qu'ils vont acheter les produits à nouveaux.